In [ ]:
%load_ext autoreload
%autoreload 2
from utils import *
import json
import re
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
recording_dir_list = [
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230329/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230330/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230331/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230401/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230402/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230403/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230404/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230405/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230406/",
    "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230407/",
]
# recording_dir_list = [
#     "../test_files/"
# ]


# Check Frame Lost


In [ ]:
get_video_creation_time("../test_files/47867_20220720_050930_MC末世生存day2.mp4")


In [ ]:
get_xml_creation_time("../test_files/47867_20220720_050930_MC末世生存day2.xml")


In [ ]:
get_file_modification_time("../test_files/47867_20220720_050930_MC末世生存day2.mp4")


In [ ]:
get_file_modification_time("../test_files/47867_20220720_050930_MC末世生存day2.xml")


In [ ]:
get_file_modification_time("testfile.flv")


In [ ]:
get_video_duration("../test_files/47867_20220720_050930_MC末世生存day2.mp4")


In [ ]:
get_video_duration("testfile.flv")


In [ ]:
get_duration_inconsistency("../test_files/47867_20220720_050930_MC末世生存day2.mp4")


In [ ]:
get_video_creation_time("/mnt/d/Videos/ApixC/apixC_MC终末旅行/transcode/12962_20230401_235504_玩MC.flv")


In [ ]:
# print duration inconsistency for all files in the directory
print("duration-xml  video-xml  file")
for dir in recording_dir_list:
    for file in os.listdir(dir):
        if file.endswith(".flv") or file.endswith(".mp4"):
            print("{:>12.1f}  {:>9.1f}  {}".format(*get_duration_inconsistency(os.path.join(dir, file)), file))


# Load Danmaku to the Pool


In [ ]:
danmaku_pool = DanmakuPool()

for dir in recording_dir_list:
    for file in os.listdir(dir):
        if file.endswith(".xml"):
            danmaku_pool.add_danmaku_from_xml(os.path.join(dir, file))
            print("added {}".format(file))


In [ ]:
danmaku_pool.df


In [ ]:
danmaku_pool = blacklist_filter(danmaku_pool, BLACKLIST)


# Examine Danmaku


In [ ]:
danmaku_pool.show_common_danmaku()


# Detect Hotspots


In [ ]:
print("kernel stdev:", KERNEL_SIGMA)
time, density = danmaku_pool.get_danmaku_density_gaussian("47867", kernel_sigma=KERNEL_SIGMA)


In [ ]:
hotspots = get_hotspots(time, density, kernel_sigma=KERNEL_SIGMA, show_plot=True, show_progress=True)
print(f"Found {len(hotspots)} hotspots")
hotspots


# Get Clips


In [ ]:
clips = get_clips(hotspots, percentile=0.75)
clips_duration = 0
for _, row in clips.iterrows():
    print(datetime.datetime.fromtimestamp(row["end"]) - datetime.datetime.fromtimestamp(row["start"]))
    clips_duration += row["end"] - row["start"]
print("Total duration:", datetime.timedelta(seconds=clips_duration))
raw_duration = time.max() - time.min()
print("Raw duration:", datetime.timedelta(seconds=raw_duration))
print("Ratio:", raw_duration / clips_duration)


# Load Videos to the Pool


In [ ]:
dir = "../test_files/"
# dir = "/mnt/d/Videos/ApixC/apixC_MC终末旅行/footages/20230403/"

video_pool = VideoPool()
for file in os.listdir(dir):
    if file.endswith(".flv") or file.endswith(".mp4"):
        video_pool.add_video(os.path.join(dir, file))
        print("added {}".format(file))
video_pool.df


# Write Clips to the Files


In [ ]:
out_dir = "../test_files/"
video_pool.generate_clips("47867", clips, out_dir, danmaku_pool=danmaku_pool)


# Whitelist Filter


In [ ]:
danmaku_pool.df


# Find Keywords


In [ ]:
keywords = [
    
]
danmaku_pool_select = danmaku_pool.whitelist_filter(keywords, "92613")
danmaku_pool_select.df


In [ ]:
danmaku_pool_select.show_common_danmaku()


In [ ]:
time, density = danmaku_pool_select.get_danmaku_density_gaussian("92613", kernel_sigma=KERNEL_SIGMA)
hotspots = get_hotspots(time, density, kernel_sigma=KERNEL_SIGMA, show_plot=True, show_progress=True)


In [ ]:
clips = get_clips(hotspots)


In [ ]:
_pool = DanmakuPool()
for _, row in clips.iterrows():
    # select danmaku from danmaku_pool between row["start"] and row["end"]
    _pool.df = pd.concat(
        [
            _pool.df,
            danmaku_pool.df[(danmaku_pool.df["time"] >= row["start"]) & (danmaku_pool.df["time"] <= row["end"])],
        ],
        ignore_index=True,
    )


In [ ]:
_pool.whitelist_filter(roomid="92613").show_common_danmaku(num=200)
